In [1]:
#数据处理
import pandas as pd
import re
import numpy as np

In [2]:
file=open('C:\\Users\\zhou\\Desktop\\word.csv')
list_line=[]
for line in file.readlines():
    line.strip()
    list_line.append(line)


In [3]:
list_line[0].strip()

'Come, Philander, let us be a marching,Every one his true love a searching,"'

In [4]:
list_word=[]
for num in range(len(list_line)):
    for i in re.split('\.| |\,|\!|\?|\"|\;',list_line[num].strip()):
        if i=='':
            pass
        else:
            list_word.append(i)

In [5]:
#one_hot编码
list_vector=[]
for i in range(len(list_word)):
    zero=np.zeros((len(list_word),1))
    zero[i]=1
    list_vector.append(zero)

In [6]:
#利用单词的前5个和后5个构建词向量

In [7]:
len(list_vector)

3533

In [8]:
list_x=[]
list_y=[]
for nu in range(5,3528):
    vector=list_vector[nu-1]+list_vector[nu-2]+list_vector[nu-3]+list_vector[nu+1]+list_vector[nu+2]+list_vector[nu+3]+list_vector[nu+4]+list_vector[nu+5]+list_vector[nu-4]+list_vector[nu-5]
    list_x.append(vector)
    list_y.append(list_vector[nu])

In [9]:
list_x[0].shape

(3533, 1)

In [186]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [187]:
def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

In [188]:
def feedback(x,y,list_w,list_a,list_z):
    list_a.reverse()
    list_a.append(x)
    list_w.reverse()
    list_z.reverse()
    sizes=len(list_w)
    hold=[]
    for nums in range(sizes):
        hold.append((list_w[nums].T).dot(sigmoid_derivative(list_z[nums])))
    grad_w=[]
    grad_b=[]
    for nums2 in range(len(hold)):
        nums2=nums2-1
        if nums2==-1:
            grad_w.append(list_a[1].dot((sigmoid_derivative(list_z[0])*(-2*(y-list_a[0]))).T))
            grad_b.append(sigmoid_derivative(list_z[0])*(-2*(y-list_a[0])))
            grad=(list_w[0].T).dot((sigmoid_derivative(list_z[0])*(-2*(y-list_a[0]))))
        if nums2!=-1:
            grad_w.append(list_a[nums2+2].dot(sigmoid_derivative(grad).T))
            grad_b.append(sigmoid_derivative(grad))
            grad=(list_w[nums2]).dot(sigmoid_derivative(grad))
    return grad_w,grad_b

In [189]:
def network(x,y):
    w1=np.random.randn(4000,3533)
    w2=np.random.randn(3533,4000)
    b1=np.random.randn(4000,1)
    b2=np.random.randn(3533,1)
    learn_rate=75
    for count in range(3):
        z1=w1.dot(x)+b1
        a1=sigmoid(z1)
        z2=w2.dot(a1)+b2
        a2=sigmoid(z2)
        list_w=[w1,w2]
        list_a=[a1,a2]
        list_z=[z1,z2]
        grad_w,grad_b=feedback(x,y,list_w,list_a,list_z)
        w1=w1-learn_rate*grad_w[-2]
        w2=w2-learn_rate*grad_w[-1]
        b1=b1-learn_rate*grad_b[-1]
        b2=b2-learn_rate*grad_b[-2]
    final_w=[w1,w2]
    final_b=[b1,b2]
    return a2,final_w,final_b
    
    

In [190]:
a2,wz,bz=network(list_x[3500],list_y[3500])

In [191]:
a2.tolist().index(max(a2.tolist()))

3505

In [192]:
len(list_x[1])

3533

In [193]:
len(list_x)

3523

In [194]:
arr_x=np.array(list_x)

In [195]:
arr_x=arr_x.reshape(3523,3533).T

In [196]:
arr_y=np.array(list_y)

In [197]:
arr_y=arr_y.reshape((3523,3533)).T

In [198]:
final,final_w,final_b=network(arr_x,arr_y)

<ipython-input-186-a6750c5d9a79>:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


In [199]:
count=4
count1=0
for i in range(len(list_x)):
    count+=1
    if final[0:,i].tolist().index(max(final[0:,i].tolist()))==count:
        count1+=1
print(count1/len(list_x))
    

1.0


In [200]:
list_sentence1=[]
for i in range(len(list_line)):
    list_sentence1.append(list_line[i].strip())

In [201]:
list_sentence=[]
for nums  in range(len(list_sentence1)):
    for i in re.split('\.|\?|\!',list_sentence1[nums]):
        if i=='':
            pass
        else:
            list_sentence.append(i)
    
    

In [202]:
list_sentence[0]

'Come, Philander, let us be a marching,Every one his true love a searching,"'

In [203]:
def pred(final_w,final_b,x_test,list_word,number):
    w1,w2=final_w
    b1,b2=final_b
    b1=np.array(b1[0:,number]).reshape((len(b1[0:,number]),1))
    b2=np.array(b2[0:,number]).reshape((len(b2[0:,number]),1))
    z1=w1.dot(x_test)+b1
    a1=sigmoid(z1)
    z2=w2.dot(a1)+b2
    y_pred=sigmoid(z2)
    return list_word[y_pred.tolist().index(max(y_pred.tolist()))]

In [204]:
pred(final_w,final_b,list_x[700],list_word,700)

<ipython-input-186-a6750c5d9a79>:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


'a'

In [205]:
def softmax(z):
    shiftz=z-np.max(z)
    return np.exp(shiftz)/sum(np.exp(shiftz))

In [206]:
def softmax_derivative(z):
    shiftz=z-np.max(z)
    return (shiftz*np.exp(shiftz)*(sum(np.exp(shiftz))-np.exp(shiftz)))/sum(np.exp(shiftz))**2

In [207]:
len(list_x[0])

3533

In [208]:
arr_x.shape

(3533, 3523)

In [209]:
def soft_network(x,y):
    w1=np.random.randn(4000,3533)
    w2=np.random.randn(3533,4000)
    learnrate=100
    for count in range(3):
        #正向传播
        z1=w1.dot(x)
        z2=w2.dot(z1)
        a=sigmoid(z2)
        #反向传播
        dy_dw2=(sigmoid_derivative(z2)*(-2*(y-a))).dot(z1.T)
        dy_dw1=(w2.T.dot(sigmoid_derivative(z2)*(-2*(y-a)))).dot(x.T)
        #更新
        w2=w2-learnrate*dy_dw2
        w1=w1-learnrate*dy_dw1
    return a,w1,w2
        
        

In [210]:
a,w1,w2=soft_network(list_x[0],list_y[0])

<ipython-input-186-a6750c5d9a79>:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


In [211]:
a.tolist().index(max(a.tolist()))

0

In [212]:
list_y[0][0:10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.]])